In [44]:
import pandas as pd
import numpy as np 
import string
import re
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer,util

In [6]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.0225026  -0.07829174 -0.02303076 ... -0.00827927  0.02652692
  -0.00201898]
 [ 0.04170236  0.0010974  -0.01553417 ... -0.02181627 -0.0635936
  -0.00875283]]


In [122]:
df = pd.read_csv('../fobokiller/data/scrapping_cleaned_top2vec.csv',index_col=0,nrows=1001)

In [123]:
df.head()

,alias,date,rate,review,review_clean
0,le-sud-paris,2018-11-20,5,"Ed W.\nElite 2021\nMiami, FL\n259\n562\n4232\n...",this is not your traditional parisian cooking ...
1,le-sud-paris,2021-07-04,2,"Jaki F.\nElite 2021\nSignal Hill, CA\n358\n270...",i'm not sure what is happening at this restaur...
2,le-sud-paris,2018-06-20,3,"Jen S.\nElite 2021\nBuffalo, NY\n71\n322\n809\...","ambience is enticing, but at the end of the da..."
3,le-sud-paris,2015-09-28,3,"Sam S.\nElite 2021\nMunich, Germany\n2133\n255...",ambience here is star but in my opinion this i...
4,le-sud-paris,2019-08-08,1,"Fahad A.\nLos Angeles, CA\n0\n10\n15\n8/8/2019...",worst restaurant ever! the servers were not th...


In [48]:
test= "Hello... I am a reviewer ! Did I love this restaurant ? Nope..."
re.split('[.?!]',test)

['Hello',
 '',
 '',
 ' I am a reviewer ',
 ' Did I love this restaurant ',
 ' Nope',
 '',
 '',
 '']

## using the whole review

In [33]:
sentences = df['review_clean']
embeddings = model.encode(sentences)


In [36]:
# Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \n {} \n {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))
    

Top-5 most similar pairs:
ambience is enticing, but at the end of the day it's definitely directed toward a tourist crowd. fortunately food is great. the interior has a make believe outdoor atmosphere with several real trees throughout the dining area. lighting is somewhat dim, seating is cosy with not the most comfortable chairs. the staff is not the most friendly, but get their jobs done well. food was awesome. went with four others and no complaints about the short menu. prices aren't cheap, so if you're on a budget or with picky eaters, probably not your first choice. see all photos from jen s. for le sud 
 ambience here is star but in my opinion this is an expensive tourist trap. the moment you enter you feel like you have been transported to the south of france but, it sort of stops there. the service is ok and arrogant. they had run out of many items on the menu by the time we arrived. the menu cards in english were a bit of a giveaway for me. it was expensive mains and many of 

## splitting the review into sentences


In [124]:
def splitter(text):
    tmp = re.split('[.?!]',text)
    tmp=filter(None, tmp)
    return list(tmp)

In [125]:
df['review_sentences'] = df['review_clean'].apply(splitter)

In [126]:
df_exploded=df.explode('review_sentences').reset_index(drop=True)

In [127]:
df_exploded

,alias,date,rate,review,review_clean,review_sentences
0,le-sud-paris,2018-11-20,5,"Ed W.\nElite 2021\nMiami, FL\n259\n562\n4232\n...",this is not your traditional parisian cooking ...,this is not your traditional parisian cooking ...
1,le-sud-paris,2018-11-20,5,"Ed W.\nElite 2021\nMiami, FL\n259\n562\n4232\n...",this is not your traditional parisian cooking ...,this restaurant is a full on southern france ...
2,le-sud-paris,2018-11-20,5,"Ed W.\nElite 2021\nMiami, FL\n259\n562\n4232\n...",this is not your traditional parisian cooking ...,we were looking for a good boulliabese
3,le-sud-paris,2018-11-20,5,"Ed W.\nElite 2021\nMiami, FL\n259\n562\n4232\n...",this is not your traditional parisian cooking ...,and here at le sud they have the boulliabese ...
4,le-sud-paris,2018-11-20,5,"Ed W.\nElite 2021\nMiami, FL\n259\n562\n4232\n...",this is not your traditional parisian cooking ...,is almost the same but thickened with egg yolks
...,...,...,...,...,...,...
8021,refuge-des-fondues-paris,2018-12-12,4,"Taylor S.\nDenver, CO\n4755\n533\n6153\n12/12/...",would i normally travel abroad to eat fondue? ...,cheese fondue is delish
8022,refuge-des-fondues-paris,2018-12-12,4,"Taylor S.\nDenver, CO\n4755\n533\n6153\n12/12/...",would i normally travel abroad to eat fondue? ...,the whole time is just a riot
8023,refuge-des-fondues-paris,2018-12-12,4,"Taylor S.\nDenver, CO\n4755\n533\n6153\n12/12/...",would i normally travel abroad to eat fondue? ...,be nice to your servers too
8024,refuge-des-fondues-paris,2018-12-12,4,"Taylor S.\nDenver, CO\n4755\n533\n6153\n12/12/...",would i normally travel abroad to eat fondue? ...,they are doing their best in a small space so...


In [128]:
sentences = df_exploded['review_sentences']
embeddings = model.encode(sentences)

In [ ]:
# Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \n {} \n {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))
    